# DeepLabCut Toolbox - DEMO (mouse reaching)
https://github.com/AlexEMG/DeepLabCut

#### The notebook accompanies the following user-guide:

Nath\*, Mathis\* et al. *Using DeepLabCut for markerless pose estimation during behavior across species* Nature Protocols, 2019: https://www.nature.com/articles/s41596-019-0176-0

This notebook starts from an already initialized project with labeled data.

**Data:** dataset is from Mathis et al. *Somatosensory Cortex Plays an Essential Role in Forelimb Motor Adaptation in Mice* Neuron, 2017: DOI:https://doi.org/10.1016/j.neuron.2017.02.049

This notebook illustrates how to:
- plot the labeled images
- train a network
- evaluate a network
- analyze a novel video
- create an automatically labeled video 
- plot the trajectories 
- identify outlier frames
- annotate the outlier frames manually
- merge the data sets and update the training set
- train a network

In [ ]:
import os
os.environ['DLClight'] = "True"
os.environ['CUDA_VISIBLE_DFVICES'] = '2'

# override PYTHONPATH to load deeplabcut module in this branch
import sys
sys.path.insert(0, os.getcwd() + '/..')
sys.path

## Import the toolbox:

In [ ]:
import deeplabcut
deeplabcut.__file__

### Set a variable to point to the project configuration file:

In [ ]:
import os
# Note that parameters of this project can be seen at: *Reaching-Mackenzie-2018-08-30/config.yaml*
from pathlib import Path

#create a variable to set the config.yaml file path:
path_config_file = os.path.join(os.getcwd(),'Reaching-Mackenzie-2018-08-30/config.yaml')
print(path_config_file)

In [ ]:
### Load trained model and data for prediction

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from deeplabcut.utils import auxiliaryfunctions
from deeplabcut.utils.auxfun_videos import imread, imresize
from deeplabcut.pose_estimation_tensorflow.config import load_config
from deeplabcut.pose_estimation_tensorflow.nnet.net_factory import pose_net
from deeplabcut.pose_estimation_tensorflow.nnet import predict
from deeplabcut.pose_estimation_tensorflow.dataset.pose_dataset import data_to_input

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

# configure experiment
config = path_config_file
shuffle = 1
trainingsetindex=1
trainFraction=.95
plotting = None
show_errors = True
comparisonbodyparts="all"
gputouse=None
rescale=False

tf.reset_default_graph()
cfg = auxiliaryfunctions.read_config(path_config_file)
modelfolder=os.path.join(cfg["project_path"],str(auxiliaryfunctions.GetModelFolder(trainFraction,shuffle,cfg)))
path_test_config = Path(modelfolder) / 'test' / 'pose_cfg.yaml'
dlc_cfg = load_config(path_test_config)

# load model
Snapshots = np.array([fn.split('.')[0]for fn in os.listdir(os.path.join(str(modelfolder), 'train'))if "index" in fn])
increasing_indices = np.argsort([int(m.split('-')[1]) for m in Snapshots])
Snapshots = Snapshots[increasing_indices]
snapshot = Snapshots[-1]

dlc_cfg['init_weights'] = os.path.join(str(modelfolder),'train',snapshot) #setting weights to corresponding snapshot.
trainingsiterations = (dlc_cfg['init_weights'].split(os.sep)[-1]).split('-')[-1] #read how many training siterations that corresponds to.
sess, inputs, outputs = predict.setup_pose_prediction(dlc_cfg)

# load data
trainingsetfolder=auxiliaryfunctions.GetTrainingSetFolder(cfg)
datafn,metadatafn=auxiliaryfunctions.GetDataandMetaDataFilenames(trainingsetfolder,trainFraction,shuffle,cfg)
Data=pd.read_hdf(os.path.join(cfg["project_path"],str(trainingsetfolder),'CollectedData_' + cfg["scorer"] + '.h5'),'df_with_missing')
data, trainIndices, testIndices, trainFraction=auxiliaryfunctions.LoadMetadata(os.path.join(cfg["project_path"],metadatafn))

### Make a Prediction and Inspect Temperature

In [ ]:
# load image
im_path = os.path.join(cfg['project_path'], Data.index[45])
image = imread(im_path, mode='RGB')
image_batch = data_to_input(image)

# execute
outputs_np = sess.run(outputs, feed_dict={inputs: image_batch})
scmap, temperature, locref = predict.extract_cnn_output(outputs_np, dlc_cfg)

# plot image and temperature
plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(temperature)
plt.axis('off')
plt.tight_layout()